In [1]:
import os
import numpy as np
from PIL import Image
import pandas as pd
import cv2
from scipy.spatial.distance import cdist  # 如果后续需要扩展其他距离计算方法

# ================================
# 目录设置
# ================================
data_dir = './videos/bed_rest_MRI_mask3'
label_dir = os.path.join(data_dir, 'label_in_png_L4L5_renamed')

# 分别用于 DSC 计算和 overlap_ratio、mask distance 计算的自动分割结果目录
auto_seg_nolap_dir_name = 'manual_prompt_frame0/SAM2_seg_mask_nolap'
auto_seg_yeslap_dir_name = 'manual_prompt_frame0/SAM2_seg_mask_yeslap'

auto_seg_nolap_dir = os.path.join(data_dir, auto_seg_nolap_dir_name)
auto_seg_yeslap_dir = os.path.join(data_dir, auto_seg_yeslap_dir_name)

# CSV 输出路径（这里以 nolap 文件夹命名）
csv_write_path = os.path.join(data_dir, auto_seg_nolap_dir_name + '_top1-1_dsc_overlap_distance.csv')

# 检查目录是否存在
for d in [label_dir, auto_seg_nolap_dir, auto_seg_yeslap_dir]:
    if not os.path.exists(d):
        raise FileNotFoundError(f"目录不存在: {d}")

# ================================
# 工具函数
# ================================

def calculate_dsc(mask1, mask2):
    """计算 Dice 系数 (DSC)"""
    intersection = np.sum((mask1 > 0) & (mask2 > 0))
    union = np.sum(mask1 > 0) + np.sum(mask2 > 0)
    return (2.0 * intersection / union) if union > 0 else 1.0


def largest_component(mask):
    """提取 mask 的最大连通区域，如果没有连通区域则返回原始 mask。"""
    mask_uint8 = mask.astype(np.uint8)
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(mask_uint8, connectivity=8)
    if num_labels <= 1:
        return mask
    # 排除背景 label=0，找出面积最大的连通区域
    largest_label = 1 + np.argmax(stats[1:, cv2.CC_STAT_AREA])
    return (labels == largest_label).astype(np.uint8)
    
def min_gap_distance(mask1, mask2):
   

    
    # 对两个 mask 分别提取最大连通区域
    largest_mask1 = largest_component(mask1)
    largest_mask2 = largest_component(mask2)
    
    # 提取边界：先腐蚀，再用原始 mask 减去腐蚀后的结果
    kernel = np.ones((3, 3), np.uint8)
    eroded1 = cv2.erode(largest_mask1, kernel, iterations=1)
    boundary1 = largest_mask1 - eroded1
    eroded2 = cv2.erode(largest_mask2, kernel, iterations=1)
    boundary2 = largest_mask2 - eroded2
    
    # 获取边界像素的坐标
    coords1 = np.argwhere(boundary1 > 0)
    coords2 = np.argwhere(boundary2 > 0)
    
    # 如果任一边界没有有效像素，则返回 0
    if coords1.size == 0 or coords2.size == 0:
        return 0
    
    # 计算两个边界上所有点之间的欧氏距离，并取最小值
    gap = cdist(coords1, coords2).min()
    return gap


# ================================
# 主程序：遍历每个标签文件，计算各项指标
# ================================
results = []

for label_file in os.listdir(label_dir):
    try:
        # 假设标签文件名为 "0.png", "1.png", … 从文件名中提取帧号
        label_index = int(label_file.split('.')[0])
    except Exception as e:
        print(f"无法解析文件名 {label_file}，跳过。")
        continue

    # 构造标签文件路径
    label_path = os.path.join(label_dir, label_file)
    
    # 构造自动分割（nolap）文件路径（用于 DSC 计算）
    nolap_file1 = f"frame_{label_index}_obj_1.png"
    nolap_file2 = f"frame_{label_index}_obj_2.png"
    nolap_path1 = os.path.join(auto_seg_nolap_dir, nolap_file1)
    nolap_path2 = os.path.join(auto_seg_nolap_dir, nolap_file2)
    
    # 构造自动分割（yeslap）文件路径（用于 overlap_ratio 与 mask distance 计算）
    yeslap_file1 = f"frame_{label_index}_obj_1.png"
    yeslap_file2 = f"frame_{label_index}_obj_2.png"
    yeslap_path1 = os.path.join(auto_seg_yeslap_dir, yeslap_file1)
    yeslap_path2 = os.path.join(auto_seg_yeslap_dir, yeslap_file2)
    
    # 检查自动分割文件是否存在
    if not (os.path.exists(nolap_path1) and os.path.exists(nolap_path2) and
            os.path.exists(yeslap_path1) and os.path.exists(yeslap_path2)):
        print(f"Frame {label_index} 的自动分割文件不存在，跳过。")
        continue

    # ---------------------------
    # DSC 计算（使用无重叠的分割结果）
    # ---------------------------
    nolap_mask1 = np.array(Image.open(nolap_path1))
    nolap_mask2 = np.array(Image.open(nolap_path2))
    
    # 按约定：类别1预测值接近 50，类别2预测值接近 100
    class1_auto_mask = (np.abs(nolap_mask1 - 50) < 10).astype(np.uint8)
    class2_auto_mask = (np.abs(nolap_mask2 - 100) < 10).astype(np.uint8)
    
    label_mask = np.array(Image.open(label_path))
    class1_label_mask = (np.abs(label_mask - 50) < 10).astype(np.uint8)
    class2_label_mask = (np.abs(label_mask - 100) < 10).astype(np.uint8)
    
    # 若尺寸不一致，则调整自动分割图像尺寸至标签图像尺寸
    target_size = (class1_label_mask.shape[1], class1_label_mask.shape[0])  # (width, height)
    class1_auto_mask = cv2.resize(class1_auto_mask, target_size, interpolation=cv2.INTER_NEAREST)
    class2_auto_mask = cv2.resize(class2_auto_mask, target_size, interpolation=cv2.INTER_NEAREST)
    
    class1_dsc = calculate_dsc(class1_label_mask, class1_auto_mask)
    class2_dsc = calculate_dsc(class2_label_mask, class2_auto_mask)
    
    class1_label_area = int(np.sum(class1_label_mask))
    class1_auto_area = int(np.sum(class1_auto_mask))
    class2_label_area = int(np.sum(class2_label_mask))
    class2_auto_area = int(np.sum(class2_auto_mask))
    
    # ---------------------------
    # Overlap Ratio 计算（使用有重叠的分割结果）
    # ---------------------------
    yeslap_mask1 = np.array(Image.open(yeslap_path1))
    yeslap_mask2 = np.array(Image.open(yeslap_path2))
    
    # 提取类别掩码：约定类别1对应值接近 50，类别2对应值接近 100
    class1_yeslap_mask = (np.abs(yeslap_mask1 - 50) < 10).astype(np.uint8)
    class2_yeslap_mask = (np.abs(yeslap_mask2 - 100) < 10).astype(np.uint8)
    
    class1_yeslap_mask = cv2.resize(class1_yeslap_mask, target_size, interpolation=cv2.INTER_NEAREST)
    class2_yeslap_mask = cv2.resize(class2_yeslap_mask, target_size, interpolation=cv2.INTER_NEAREST)
    
    intersection_yeslap = np.sum((class1_yeslap_mask > 0) & (class2_yeslap_mask > 0))
    area_sum_yeslap = np.sum(class1_yeslap_mask > 0) + np.sum(class2_yeslap_mask > 0)
    overlap_ratio = (intersection_yeslap / area_sum_yeslap) if area_sum_yeslap > 0 else 0

    # ---------------------------
    # Mask Distance 计算（使用有重叠的分割结果）
    # ---------------------------
    # 计算类别1与类别2之间的最小间隙距离（单位：像素）
    mask_distance = min_gap_distance(class1_yeslap_mask, class2_yeslap_mask)
    
    # ---------------------------
    # 保存当前帧的各项指标
    # ---------------------------
    file_results = {
        "file_name": label_file,
        "class1_dsc": class1_dsc,
        "class1_label_area": class1_label_area,
        "class1_auto_area": class1_auto_area,
        "class2_dsc": class2_dsc,
        "class2_label_area": class2_label_area,
        "class2_auto_area": class2_auto_area,
        "overlap_ratio": overlap_ratio,
        "mask_distance": mask_distance
    }
    results.append(file_results)
    print(f"Processed {label_file}: Class1 DSC={class1_dsc:.3f}, Class2 DSC={class2_dsc:.3f}, "
          f"Overlap Ratio={overlap_ratio:.3f}, Mask Distance={mask_distance:.3f}")

# 保存结果到 CSV 文件
results_df = pd.DataFrame(results)
results_df.to_csv(csv_write_path, index=False)
print(f"Results saved to {csv_write_path}.")


Processed 00071.png: Class1 DSC=0.745, Class2 DSC=0.874, Overlap Ratio=0.002, Mask Distance=0.000
Processed 00049.png: Class1 DSC=0.805, Class2 DSC=0.926, Overlap Ratio=0.254, Mask Distance=0.000
Processed 00037.png: Class1 DSC=0.750, Class2 DSC=0.296, Overlap Ratio=0.448, Mask Distance=0.000
Processed 00038.png: Class1 DSC=0.894, Class2 DSC=0.930, Overlap Ratio=0.005, Mask Distance=0.000
Processed 00028.png: Class1 DSC=0.947, Class2 DSC=0.877, Overlap Ratio=0.009, Mask Distance=0.000
Processed 00066.png: Class1 DSC=0.785, Class2 DSC=0.942, Overlap Ratio=0.000, Mask Distance=1.000
Processed 00068.png: Class1 DSC=0.071, Class2 DSC=0.770, Overlap Ratio=0.000, Mask Distance=1.000
Processed 00014.png: Class1 DSC=0.955, Class2 DSC=0.872, Overlap Ratio=0.004, Mask Distance=0.000
Processed 00060.png: Class1 DSC=0.494, Class2 DSC=0.704, Overlap Ratio=0.314, Mask Distance=0.000
Processed 00053.png: Class1 DSC=0.016, Class2 DSC=0.538, Overlap Ratio=0.001, Mask Distance=0.000
Processed 00000.png:

In [2]:
import os
import numpy as np
from PIL import Image
import pandas as pd
import cv2

# 定义目录（请根据实际情况修改）
label_dir = os.path.join(data_dir, 'label_in_png_L4L5_renamed')
# 自动分割结果所在目录（这里选用去除重叠版）
auto_seg_dir_name = 'manual_prompt_frame0/multimask_nolap'
auto_seg_dir = os.path.join(data_dir, auto_seg_dir_name)
csv_write_path = os.path.join(data_dir, auto_seg_dir_name + '_9pair_dsc.csv')

# 检查目录是否存在
if not os.path.exists(label_dir) or not os.path.exists(auto_seg_dir):
    raise FileNotFoundError("标签目录或自动分割目录不存在！")

# 定义计算 DSC 的函数
def calculate_dsc(mask1, mask2):
    intersection = np.sum((mask1 > 0) & (mask2 > 0))
    union = np.sum(mask1 > 0) + np.sum(mask2 > 0)
    return (2.0 * intersection / union) if union > 0 else 1.0

# 初始化结果列表
results = []

# 遍历标签目录中的每个标签文件
for label_file in os.listdir(label_dir):
    # 从标签文件名中提取帧号（假设文件名为 "0.png", "1.png", ...）
    try:
        label_index = int(label_file.split('.')[0])
    except Exception as e:
        print(f"无法解析文件名 {label_file}，跳过。")
        continue

    # 跳过 frame_index 为 0 的帧
    if label_index == 0:
        continue

    # 构造标签文件的完整路径
    label_path = os.path.join(label_dir, label_file)
    
    # 从自动分割目录中查找当前帧的候选文件
    # 要求文件名以 "frame_{label_index}_" 开头且包含 "nolap"
    candidate_files = [f for f in os.listdir(auto_seg_dir)
                       if f.startswith(f"frame_{label_index}_") and "nolap" in f]
    
    # 如果候选文件数不为 9，则跳过该帧
    if len(candidate_files) != 9:
        print(f"Frame {label_index}: 预期候选文件 9 个，实际 {len(candidate_files)} 个，跳过。")
        continue

    # 读取标签图像
    label_mask = np.array(Image.open(label_path))
    
    # 对标签图像生成对应的类别二值 mask
    # 约定：标签中类别1的像素值接近 50，类别2的像素值接近 100
    class1_label_mask = (np.abs(label_mask - 50) < 10).astype(np.uint8)
    class2_label_mask = (np.abs(label_mask - 100) < 10).astype(np.uint8)

    # 遍历该帧的每个候选文件
    for candidate_file in candidate_files:
        candidate_path = os.path.join(auto_seg_dir, candidate_file)
        pred_mask = np.array(Image.open(candidate_path))
        
        # 对自动分割的候选组合，提取类别预测：
        # 约定：预测值接近 100 表示类别1（obj1），接近 200 表示类别2（obj2）
        class1_pred_mask = (np.abs(pred_mask - 100) < 10).astype(np.uint8)
        class2_pred_mask = (np.abs(pred_mask - 200) < 10).astype(np.uint8)
        
        # 若预测 mask 尺寸与标签不一致，则用最近邻插值调整
        target_size = (class1_label_mask.shape[1], class1_label_mask.shape[0])  # (width, height)
        class1_pred_mask = cv2.resize(class1_pred_mask, target_size, interpolation=cv2.INTER_NEAREST)
        class2_pred_mask = cv2.resize(class2_pred_mask, target_size, interpolation=cv2.INTER_NEAREST)
        
        # 计算 DSC
        class1_dsc = calculate_dsc(class1_label_mask, class1_pred_mask)
        class2_dsc = calculate_dsc(class2_label_mask, class2_pred_mask)
        
        # 记录当前候选文件的结果
        result = {
            "file_name": label_file,
            "frame_index": label_index,
            "candidate_file": candidate_file,
            "class1_dsc": class1_dsc,
            "class2_dsc": class2_dsc,
            "class1_label_area": int(np.sum(class1_label_mask)),
            "class1_pred_area": int(np.sum(class1_pred_mask)),
            "class2_label_area": int(np.sum(class2_label_mask)),
            "class2_pred_area": int(np.sum(class2_pred_mask))
        }
        results.append(result)
        print(f"Frame {label_index}, {candidate_file}: Class1 DSC = {class1_dsc:.3f}, Class2 DSC = {class2_dsc:.3f}")

# 保存结果到 CSV 文件
results_df = pd.DataFrame(results)
results_df.to_csv(csv_write_path, index=False)
print(f"结果已保存到 {csv_write_path}.")


Frame 71, frame_71_obj1_top1_obj2_top3_nolap.png: Class1 DSC = 0.747, Class2 DSC = 0.001
Frame 71, frame_71_obj1_top3_obj2_top2_nolap.png: Class1 DSC = 0.000, Class2 DSC = 0.634
Frame 71, frame_71_obj1_top2_obj2_top1_nolap.png: Class1 DSC = 0.806, Class2 DSC = 0.882
Frame 71, frame_71_obj1_top2_obj2_top2_nolap.png: Class1 DSC = 0.740, Class2 DSC = 0.689
Frame 71, frame_71_obj1_top3_obj2_top3_nolap.png: Class1 DSC = 0.000, Class2 DSC = 0.000
Frame 71, frame_71_obj1_top2_obj2_top3_nolap.png: Class1 DSC = 0.658, Class2 DSC = 0.001
Frame 71, frame_71_obj1_top1_obj2_top2_nolap.png: Class1 DSC = 0.699, Class2 DSC = 0.764
Frame 71, frame_71_obj1_top1_obj2_top1_nolap.png: Class1 DSC = 0.744, Class2 DSC = 0.859
Frame 71, frame_71_obj1_top3_obj2_top1_nolap.png: Class1 DSC = 0.000, Class2 DSC = 0.856
Frame 49, frame_49_obj1_top2_obj2_top1_nolap.png: Class1 DSC = 0.094, Class2 DSC = 0.849
Frame 49, frame_49_obj1_top2_obj2_top3_nolap.png: Class1 DSC = 0.096, Class2 DSC = 0.000
Frame 49, frame_49_ob

Frame 62, frame_62_obj1_top2_obj2_top1_nolap.png: Class1 DSC = 0.803, Class2 DSC = 0.781
Frame 62, frame_62_obj1_top2_obj2_top3_nolap.png: Class1 DSC = 0.651, Class2 DSC = 0.000
Frame 62, frame_62_obj1_top1_obj2_top1_nolap.png: Class1 DSC = 0.848, Class2 DSC = 0.888
Frame 62, frame_62_obj1_top1_obj2_top2_nolap.png: Class1 DSC = 0.849, Class2 DSC = 0.862
Frame 62, frame_62_obj1_top1_obj2_top3_nolap.png: Class1 DSC = 0.847, Class2 DSC = 0.000
Frame 62, frame_62_obj1_top3_obj2_top1_nolap.png: Class1 DSC = 0.000, Class2 DSC = 0.888
Frame 62, frame_62_obj1_top2_obj2_top2_nolap.png: Class1 DSC = 0.778, Class2 DSC = 0.648
Frame 58, frame_58_obj1_top1_obj2_top1_nolap.png: Class1 DSC = 0.781, Class2 DSC = 0.917
Frame 58, frame_58_obj1_top1_obj2_top2_nolap.png: Class1 DSC = 0.764, Class2 DSC = 0.882
Frame 58, frame_58_obj1_top3_obj2_top1_nolap.png: Class1 DSC = 0.018, Class2 DSC = 0.923
Frame 58, frame_58_obj1_top2_obj2_top3_nolap.png: Class1 DSC = 0.038, Class2 DSC = 0.000
Frame 58, frame_58_ob

Frame 57, frame_57_obj1_top1_obj2_top3_nolap.png: Class1 DSC = 0.803, Class2 DSC = 0.000
Frame 57, frame_57_obj1_top1_obj2_top1_nolap.png: Class1 DSC = 0.828, Class2 DSC = 0.929
Frame 57, frame_57_obj1_top3_obj2_top1_nolap.png: Class1 DSC = 0.002, Class2 DSC = 0.926
Frame 57, frame_57_obj1_top2_obj2_top2_nolap.png: Class1 DSC = 0.033, Class2 DSC = 0.612
Frame 57, frame_57_obj1_top1_obj2_top2_nolap.png: Class1 DSC = 0.754, Class2 DSC = 0.840
Frame 57, frame_57_obj1_top2_obj2_top1_nolap.png: Class1 DSC = 0.033, Class2 DSC = 0.931
Frame 56, frame_56_obj1_top3_obj2_top2_nolap.png: Class1 DSC = 0.000, Class2 DSC = 0.752
Frame 56, frame_56_obj1_top2_obj2_top2_nolap.png: Class1 DSC = 0.126, Class2 DSC = 0.753
Frame 56, frame_56_obj1_top2_obj2_top1_nolap.png: Class1 DSC = 0.136, Class2 DSC = 0.788
Frame 56, frame_56_obj1_top2_obj2_top3_nolap.png: Class1 DSC = 0.135, Class2 DSC = 0.000
Frame 56, frame_56_obj1_top3_obj2_top3_nolap.png: Class1 DSC = 0.000, Class2 DSC = 0.000
Frame 56, frame_56_ob

Frame 32, frame_32_obj1_top1_obj2_top3_nolap.png: Class1 DSC = 0.774, Class2 DSC = 0.000
Frame 32, frame_32_obj1_top3_obj2_top3_nolap.png: Class1 DSC = 0.000, Class2 DSC = 0.000
Frame 32, frame_32_obj1_top3_obj2_top2_nolap.png: Class1 DSC = 0.000, Class2 DSC = 0.892
Frame 15, frame_15_obj1_top2_obj2_top1_nolap.png: Class1 DSC = 0.951, Class2 DSC = 0.874
Frame 15, frame_15_obj1_top1_obj2_top2_nolap.png: Class1 DSC = 0.897, Class2 DSC = 0.903
Frame 15, frame_15_obj1_top2_obj2_top3_nolap.png: Class1 DSC = 0.950, Class2 DSC = 0.007
Frame 15, frame_15_obj1_top1_obj2_top3_nolap.png: Class1 DSC = 0.896, Class2 DSC = 0.007
Frame 15, frame_15_obj1_top3_obj2_top2_nolap.png: Class1 DSC = 0.014, Class2 DSC = 0.903
Frame 15, frame_15_obj1_top1_obj2_top1_nolap.png: Class1 DSC = 0.897, Class2 DSC = 0.874
Frame 15, frame_15_obj1_top2_obj2_top2_nolap.png: Class1 DSC = 0.951, Class2 DSC = 0.903
Frame 15, frame_15_obj1_top3_obj2_top1_nolap.png: Class1 DSC = 0.015, Class2 DSC = 0.869
Frame 15, frame_15_ob

In [3]:
df = pd.read_csv(csv_write_path)
df.head()

file_name  frame_index                          candidate_file  class1_dsc  \
0  00071.png           71  frame_71_obj1_top1_obj2_top3_nolap.png    0.746548   
1  00071.png           71  frame_71_obj1_top3_obj2_top2_nolap.png    0.000000   
2  00071.png           71  frame_71_obj1_top2_obj2_top1_nolap.png    0.806279   
3  00071.png           71  frame_71_obj1_top2_obj2_top2_nolap.png    0.739533   
4  00071.png           71  frame_71_obj1_top3_obj2_top3_nolap.png    0.000000   

   class2_dsc  class1_label_area  class1_pred_area  class2_label_area  \
0    0.001478               1074              1388                734   
1    0.634170               1074              1523                734   
2    0.882184               1074              1538                734   
3    0.689283               1074              1625                734   
4    0.000000               1074              1333                734   

   class2_pred_area  
0               619  
1              1297  
2               658  
3               563  
4               647

In [4]:
import pandas as pd

# 读取 CSV 文件
df = pd.read_csv(csv_write_path)

# 新建一列 total_dsc，表示 class1_dsc 与 class2_dsc 之和
df['total_dsc'] = df['class1_dsc'] + df['class2_dsc']

# 如果需要排除 frame_index==0 的情况，可以过滤掉
df = df[df['frame_index'] != 0]

# 用于保存每个 frame_index 的比较结果
results = []

# 按 frame_index 分组处理
for frame_idx, group in df.groupby('frame_index'):
    # 在当前帧中找到 DSC 总和最大的那一行
    max_row = group.loc[group['total_dsc'].idxmax()]
    
    # 在当前帧中查找候选文件名中同时包含 "obj1_top1" 和 "obj2_top1"
    top1_group = group[
        group['candidate_file'].str.contains("obj1_top1") &
        group['candidate_file'].str.contains("obj2_top1")
    ]
    
    if top1_group.empty:
        print(f"Frame {frame_idx}: 没有找到候选文件名包含 'obj1_top1' 和 'obj2_top1' 的记录，跳过。")
        continue
    else:
        # 如果有多个，取第一个
        top1_row = top1_group.iloc[0]
    
    # 计算 DSC 总和的差值：最佳 DSC 与 top1_top1 DSC 之间的差值
    dsc_diff = max_row['total_dsc'] - top1_row['total_dsc']
    
    results.append({
        'frame_index': frame_idx,
        'max_candidate_file': max_row['candidate_file'],
        'max_total_dsc': max_row['total_dsc'],
        'top1_candidate_file': top1_row['candidate_file'],
        'top1_total_dsc': top1_row['total_dsc'],
        'dsc_diff': dsc_diff
    })

# 将结果转换为 DataFrame 并打印
result_df = pd.DataFrame(results)
print(result_df)
dif_csv_write_path = os.path.join(data_dir, auto_seg_dir_name +'_compare_top_candidates.csv')
# 如果需要将结果保存到 CSV 文件，可以取消下面的注释：
result_df.to_csv(dif_csv_write_path , index=False)


    frame_index                      max_candidate_file  max_total_dsc  \
0             1   frame_1_obj1_top1_obj2_top1_nolap.png       1.811395   
1             2   frame_2_obj1_top2_obj2_top2_nolap.png       1.879512   
2             3   frame_3_obj1_top1_obj2_top1_nolap.png       1.890837   
3             4   frame_4_obj1_top2_obj2_top1_nolap.png       1.881600   
4             5   frame_5_obj1_top2_obj2_top1_nolap.png       1.675950   
..          ...                                     ...            ...   
66           67  frame_67_obj1_top2_obj2_top1_nolap.png       1.744927   
67           68  frame_68_obj1_top2_obj2_top2_nolap.png       1.393086   
68           69  frame_69_obj1_top1_obj2_top1_nolap.png       1.405860   
69           70  frame_70_obj1_top2_obj2_top2_nolap.png       1.545162   
70           71  frame_71_obj1_top2_obj2_top1_nolap.png       1.688463   

                       top1_candidate_file  top1_total_dsc  dsc_diff  
0    frame_1_obj1_top1_obj2_top1_nolap.p

In [11]:
import pandas as pd

# 读取 CSV 文件
csv_write_path = os.path.join(data_dir, 
                              'manual_prompt_frame0/multimask_nolap_compare_top_candidates_with_ratio.csv')
df = pd.read_csv(csv_write_path)
print(df.shape)
df.head()

(71, 8)


frame_index                     max_candidate_file  max_total_dsc  \
0            1  frame_1_obj1_top1_obj2_top1_nolap.png       1.811395   
1            2  frame_2_obj1_top2_obj2_top2_nolap.png       1.879512   
2            3  frame_3_obj1_top1_obj2_top1_nolap.png       1.890837   
3            4  frame_4_obj1_top2_obj2_top1_nolap.png       1.881600   
4            5  frame_5_obj1_top2_obj2_top1_nolap.png       1.675950   

                     top1_candidate_file  top1_total_dsc  dsc_diff  \
0  frame_1_obj1_top1_obj2_top1_nolap.png        1.811395  0.000000   
1  frame_2_obj1_top1_obj2_top1_nolap.png        1.829573  0.049939   
2  frame_3_obj1_top1_obj2_top1_nolap.png        1.890837  0.000000   
3  frame_4_obj1_top1_obj2_top1_nolap.png        1.846460  0.035139   
4  frame_5_obj1_top1_obj2_top1_nolap.png        1.572533  0.103417   

   overlap_ratio  mask_distance  
0       0.006072              0  
1       0.018538              0  
2       0.005647              0  
3       0.004100              0  
4       0.011144              0

In [6]:
import pandas as pd
import numpy as np

# 假设 df 已经加载，df 包含列：
# 'frame_index', 'max_candidate_file', 'max_total_dsc', 'top1_candidate_file', 
# 'top1_total_dsc', 'dsc_diff', 'overlap_ratio', 'mask_distance'

# 1. 计算 overlap_ratio 的 90 百分位值（即 top10% 的阈值）
overlap_threshold = np.percentile(df['overlap_ratio'], 80)
print("overlap_ratio 的 90 百分位值（阈值）为:", overlap_threshold)

# 2. 用户手动设置 mask_distance 的最小阈值（请根据实际情况设置）
min_mask_distance = 0.5  # 例如设为 0.5，单位同 df 中 mask_distance 的单位
print("手动设置的 mask_distance 最小阈值为:", min_mask_distance)

# 3. 筛选出 overlap_ratio 大于等于 overlap_threshold 且 mask_distance 大于等于 min_mask_distance 的所有行
filtered_df = df[(df['overlap_ratio'] >= overlap_threshold) | (df['mask_distance'] >= min_mask_distance)]

if filtered_df.empty:
    print("没有满足条件的行！")
else:
    # 计算所有筛选行的 abs(dsc_diff) 之和
    abs_dsc_diff_sum = np.abs(filtered_df['dsc_diff']).sum()
    # 计算筛选行数
    row_count = filtered_df.shape[0]
    # 计算平均绝对 dsc_diff
    avg_abs_dsc_diff = abs_dsc_diff_sum / row_count
    
    print("满足条件的行数:", row_count)
    print("绝对 dsc_diff 之和:", abs_dsc_diff_sum)
    print("平均绝对 dsc_diff:", avg_abs_dsc_diff)


overlap_ratio 的 90 百分位值（阈值）为: 0.211405529953917
手动设置的 mask_distance 最小阈值为: 0.5
满足条件的行数: 20
绝对 dsc_diff 之和: 7.657908956757763
平均绝对 dsc_diff: 0.38289544783788815


In [7]:
filtered_df

frame_index                      max_candidate_file  max_total_dsc  \
26           27  frame_27_obj1_top1_obj2_top2_nolap.png       1.698330   
32           33  frame_33_obj1_top1_obj2_top1_nolap.png       1.760567   
35           36  frame_36_obj1_top2_obj2_top2_nolap.png       1.466622   
36           37  frame_37_obj1_top2_obj2_top2_nolap.png       1.890511   
38           39  frame_39_obj1_top2_obj2_top1_nolap.png       1.820119   
46           47  frame_47_obj1_top1_obj2_top2_nolap.png       1.655461   
48           49  frame_49_obj1_top1_obj2_top1_nolap.png       1.720567   
49           50  frame_50_obj1_top1_obj2_top2_nolap.png       1.508913   
50           51  frame_51_obj1_top2_obj2_top2_nolap.png       1.594888   
54           55  frame_55_obj1_top2_obj2_top2_nolap.png       1.619963   
58           59  frame_59_obj1_top2_obj2_top2_nolap.png       1.693814   
59           60  frame_60_obj1_top1_obj2_top2_nolap.png       1.786586   
60           61  frame_61_obj1_top1_obj2_top2_nolap.png       1.700498   
62           63  frame_63_obj1_top1_obj2_top2_nolap.png       1.719227   
63           64  frame_64_obj1_top1_obj2_top1_nolap.png       1.671613   
65           66  frame_66_obj1_top1_obj2_top1_nolap.png       1.715412   
66           67  frame_67_obj1_top2_obj2_top1_nolap.png       1.744927   
67           68  frame_68_obj1_top2_obj2_top2_nolap.png       1.393086   
68           69  frame_69_obj1_top1_obj2_top1_nolap.png       1.405860   
69           70  frame_70_obj1_top2_obj2_top2_nolap.png       1.545162   

                       top1_candidate_file  top1_total_dsc  dsc_diff  \
26  frame_27_obj1_top1_obj2_top1_nolap.png        1.491618  0.206712   
32  frame_33_obj1_top1_obj2_top1_nolap.png        1.760567  0.000000   
35  frame_36_obj1_top1_obj2_top1_nolap.png        1.195967  0.270655   
36  frame_37_obj1_top1_obj2_top1_nolap.png        0.989911  0.900600   
38  frame_39_obj1_top1_obj2_top1_nolap.png        1.145538  0.674581   
46  frame_47_obj1_top1_obj2_top1_nolap.png        1.655103  0.000359   
48  frame_49_obj1_top1_obj2_top1_nolap.png        1.720567  0.000000   
49  frame_50_obj1_top1_obj2_top1_nolap.png        0.759097  0.749816   
50  frame_51_obj1_top1_obj2_top1_nolap.png        0.881133  0.713755   
54  frame_55_obj1_top1_obj2_top1_nolap.png        0.858613  0.761351   
58  frame_59_obj1_top1_obj2_top1_nolap.png        0.725385  0.968429   
59  frame_60_obj1_top1_obj2_top1_nolap.png        1.201145  0.585441   
60  frame_61_obj1_top1_obj2_top1_nolap.png        1.349524  0.350974   
62  frame_63_obj1_top1_obj2_top1_nolap.png        1.439946  0.279281   
63  frame_64_obj1_top1_obj2_top1_nolap.png        1.671613  0.000000   
65  frame_66_obj1_top1_obj2_top1_nolap.png        1.715412  0.000000   
66  frame_67_obj1_top1_obj2_top1_nolap.png        1.740862  0.004065   
67  frame_68_obj1_top1_obj2_top1_nolap.png        0.842946  0.550140   
68  frame_69_obj1_top1_obj2_top1_nolap.png        1.405860  0.000000   
69  frame_70_obj1_top1_obj2_top1_nolap.png        0.903414  0.641749   

    overlap_ratio  mask_distance  
26       0.374157              0  
32       0.212610              0  
35       0.475282              0  
36       0.447810              0  
38       0.240619              0  
46       0.211406              0  
48       0.253537              0  
49       0.342604              0  
50       0.327384              0  
54       0.000364              1  
58       0.000000              1  
59       0.314329              0  
60       0.370415              0  
62       0.386716              0  
63       0.213694              0  
65       0.000000              1  
66       0.000000              1  
67       0.000000              1  
68       0.222414              0  
69       0.489906              0

In [8]:
遍历df，如果df的frame_index在filtered_df有对应的行，就取其对应行的max_candidate_file
选取df中的candidate_file的字符串等于max_candidate_file的那一行
如果df的frame_index在filtered_df没有对应的行，就选取df中的candidate_file字符串中
frame_idx对应的_obj1是top1且obj2是top1的那行

SyntaxError: invalid character '，' (U+FF0C) (1205223928.py, line 1)

In [12]:
filtered_df

frame_index                      max_candidate_file  max_total_dsc  \
26           27  frame_27_obj1_top1_obj2_top2_nolap.png       1.698330   
32           33  frame_33_obj1_top1_obj2_top1_nolap.png       1.760567   
35           36  frame_36_obj1_top2_obj2_top2_nolap.png       1.466622   
36           37  frame_37_obj1_top2_obj2_top2_nolap.png       1.890511   
38           39  frame_39_obj1_top2_obj2_top1_nolap.png       1.820119   
46           47  frame_47_obj1_top1_obj2_top2_nolap.png       1.655461   
48           49  frame_49_obj1_top1_obj2_top1_nolap.png       1.720567   
49           50  frame_50_obj1_top1_obj2_top2_nolap.png       1.508913   
50           51  frame_51_obj1_top2_obj2_top2_nolap.png       1.594888   
54           55  frame_55_obj1_top2_obj2_top2_nolap.png       1.619963   
58           59  frame_59_obj1_top2_obj2_top2_nolap.png       1.693814   
59           60  frame_60_obj1_top1_obj2_top2_nolap.png       1.786586   
60           61  frame_61_obj1_top1_obj2_top2_nolap.png       1.700498   
62           63  frame_63_obj1_top1_obj2_top2_nolap.png       1.719227   
63           64  frame_64_obj1_top1_obj2_top1_nolap.png       1.671613   
65           66  frame_66_obj1_top1_obj2_top1_nolap.png       1.715412   
66           67  frame_67_obj1_top2_obj2_top1_nolap.png       1.744927   
67           68  frame_68_obj1_top2_obj2_top2_nolap.png       1.393086   
68           69  frame_69_obj1_top1_obj2_top1_nolap.png       1.405860   
69           70  frame_70_obj1_top2_obj2_top2_nolap.png       1.545162   

                       top1_candidate_file  top1_total_dsc  dsc_diff  \
26  frame_27_obj1_top1_obj2_top1_nolap.png        1.491618  0.206712   
32  frame_33_obj1_top1_obj2_top1_nolap.png        1.760567  0.000000   
35  frame_36_obj1_top1_obj2_top1_nolap.png        1.195967  0.270655   
36  frame_37_obj1_top1_obj2_top1_nolap.png        0.989911  0.900600   
38  frame_39_obj1_top1_obj2_top1_nolap.png        1.145538  0.674581   
46  frame_47_obj1_top1_obj2_top1_nolap.png        1.655103  0.000359   
48  frame_49_obj1_top1_obj2_top1_nolap.png        1.720567  0.000000   
49  frame_50_obj1_top1_obj2_top1_nolap.png        0.759097  0.749816   
50  frame_51_obj1_top1_obj2_top1_nolap.png        0.881133  0.713755   
54  frame_55_obj1_top1_obj2_top1_nolap.png        0.858613  0.761351   
58  frame_59_obj1_top1_obj2_top1_nolap.png        0.725385  0.968429   
59  frame_60_obj1_top1_obj2_top1_nolap.png        1.201145  0.585441   
60  frame_61_obj1_top1_obj2_top1_nolap.png        1.349524  0.350974   
62  frame_63_obj1_top1_obj2_top1_nolap.png        1.439946  0.279281   
63  frame_64_obj1_top1_obj2_top1_nolap.png        1.671613  0.000000   
65  frame_66_obj1_top1_obj2_top1_nolap.png        1.715412  0.000000   
66  frame_67_obj1_top1_obj2_top1_nolap.png        1.740862  0.004065   
67  frame_68_obj1_top1_obj2_top1_nolap.png        0.842946  0.550140   
68  frame_69_obj1_top1_obj2_top1_nolap.png        1.405860  0.000000   
69  frame_70_obj1_top1_obj2_top1_nolap.png        0.903414  0.641749   

    overlap_ratio  mask_distance  
26       0.374157              0  
32       0.212610              0  
35       0.475282              0  
36       0.447810              0  
38       0.240619              0  
46       0.211406              0  
48       0.253537              0  
49       0.342604              0  
50       0.327384              0  
54       0.000364              1  
58       0.000000              1  
59       0.314329              0  
60       0.370415              0  
62       0.386716              0  
63       0.213694              0  
65       0.000000              1  
66       0.000000              1  
67       0.000000              1  
68       0.222414              0  
69       0.489906              0

In [16]:
csv_write_path = os.path.join(data_dir, auto_seg_dir_name + '_9pair_dsc.csv')
df = pd.read_csv(csv_write_path)
df.shape


# 假设 df 和 filtered_df 已经加载为 DataFrame 对象
# df 的列包括：file_name, frame_index, candidate_file, class1_dsc, class2_dsc, ...
# filtered_df 的列包括：frame_index, max_candidate_file, max_total_dsc, top1_candidate_file, top1_total_dsc, dsc_diff, overlap_ratio, mask_distance

selected_rows = []

# 按 frame_index 对 df 分组
for frame, group in df.groupby('frame_index'):
    # 在 filtered_df 中查找当前 frame 的记录
    filtered = filtered_df[filtered_df['frame_index'] == frame]
    
    if not filtered.empty:
        # 如果存在，则取该行的 max_candidate_file
        max_candidate_file = filtered.iloc[0]['max_candidate_file']
        # 在 group 中选取 candidate_file 等于 max_candidate_file 的那一行
        selected = group[group['candidate_file'] == max_candidate_file]
        if not selected.empty:
            selected_rows.append(selected.iloc[0])
        else:
            # 如果未找到，则作为后备选择，选取包含 "obj1_top1" 和 "obj2_top1" 的记录
            fallback = group[group['candidate_file'].str.contains("obj1_top1") & 
                             group['candidate_file'].str.contains("obj2_top1")]
            if not fallback.empty:
                selected_rows.append(fallback.iloc[0])
    else:
        # 如果 filtered_df 中没有该 frame 的记录，则选取 group 中 candidate_file 同时包含 "obj1_top1" 和 "obj2_top1" 的记录
        fallback = group[group['candidate_file'].str.contains("obj1_top1") & 
                         group['candidate_file'].str.contains("obj2_top1")]
        if not fallback.empty:
            selected_rows.append(fallback.iloc[0])

# 将选中的行合并为新的 DataFrame
selected_df = pd.DataFrame(selected_rows)

# 输出结果
print(selected_df.shape)
selected_df.head()

(71, 9)


file_name  frame_index                         candidate_file  \
307  00001.png            1  frame_1_obj1_top1_obj2_top1_nolap.png   
414  00002.png            2  frame_2_obj1_top1_obj2_top1_nolap.png   
272  00003.png            3  frame_3_obj1_top1_obj2_top1_nolap.png   
424  00004.png            4  frame_4_obj1_top1_obj2_top1_nolap.png   
531  00005.png            5  frame_5_obj1_top1_obj2_top1_nolap.png   

     class1_dsc  class2_dsc  class1_label_area  class1_pred_area  \
307    0.912786    0.898608               1264              1442   
414    0.919310    0.910263                877               920   
272    0.953103    0.937733               1432              1468   
424    0.885213    0.961248               1342              1620   
531    0.739200    0.833333               1299              1826   

     class2_label_area  class2_pred_area  
307                809               700  
414               1087               930  
272                792               814  
424               1058              1058  
531                809               835

In [17]:
import pandas as pd
import numpy as np
from scipy.stats import ks_2samp

# 假设 selected_df 已加载
# 示例 selected_df 如下：
#   file_name  frame_index                    candidate_file  class1_dsc  class2_dsc  class1_label_area  class1_pred_area  class2_label_area  class2_pred_area
# 307   00001.png            1   frame_1_obj1_top1_obj2_top1_nolap.png    0.912786    0.898608               1264              1442               809              700
# 414   00002.png            2   frame_2_obj1_top1_obj2_top1_nolap.png    0.919310    0.910263                877               920              1087              930
# 272   00003.png            3   frame_3_obj1_top1_obj2_top1_nolap.png    0.953103    0.937733               1432              1468               792              814
# 424   00004.png            4   frame_4_obj1_top1_obj2_top1_nolap.png    0.885213    0.961248               1342              1620              1058             1058
# 531   00005.png            5   frame_5_obj1_top1_obj2_top1_nolap.png    0.739200    0.833333               1299              1826               809              835

# ---------------------------
# 1. 计算每个类别 DSC 的均值与标准差
# ---------------------------
class1_dsc_mean = selected_df['class1_dsc'].mean()
class1_dsc_std  = selected_df['class1_dsc'].std()

class2_dsc_mean = selected_df['class2_dsc'].mean()
class2_dsc_std  = selected_df['class2_dsc'].std()

print("Class1 DSC: mean = {:.3f}, std = {:.3f}".format(class1_dsc_mean, class1_dsc_std))
print("Class2 DSC: mean = {:.3f}, std = {:.3f}".format(class2_dsc_mean, class2_dsc_std))

# ---------------------------
# 2. 统计预测区域面积和标签区域面积
# ---------------------------
# 对类别1
class1_label = selected_df['class1_label_area']
class1_pred  = selected_df['class1_pred_area']

# 对类别2
class2_label = selected_df['class2_label_area']
class2_pred  = selected_df['class2_pred_area']

# 计算均值和标准差（如果需要）
class1_label_mean = class1_label.mean()
class1_label_std  = class1_label.std()
class1_pred_mean  = class1_pred.mean()
class1_pred_std   = class1_pred.std()

class2_label_mean = class2_label.mean()
class2_label_std  = class2_label.std()
class2_pred_mean  = class2_pred.mean()
class2_pred_std   = class2_pred.std()

print("\nClass1 Area:")
print("  Label: mean = {:.1f}, std = {:.1f}".format(class1_label_mean, class1_label_std))
print("  Pred:  mean = {:.1f}, std = {:.1f}".format(class1_pred_mean, class1_pred_std))
print("Class2 Area:")
print("  Label: mean = {:.1f}, std = {:.1f}".format(class2_label_mean, class2_label_std))
print("  Pred:  mean = {:.1f}, std = {:.1f}".format(class2_pred_mean, class2_pred_std))

# ---------------------------
# 3. 使用 KS test 检验预测面积和标签面积是否存在统计学差异
# ---------------------------
ks_class1 = ks_2samp(class1_label, class1_pred)
ks_class2 = ks_2samp(class2_label, class2_pred)

print("\nKS Test for Class1 Area:")
print("  Statistic = {:.4f}, p-value = {:.4f}".format(ks_class1.statistic, ks_class1.pvalue))
print("KS Test for Class2 Area:")
print("  Statistic = {:.4f}, p-value = {:.4f}".format(ks_class2.statistic, ks_class2.pvalue))


Class1 DSC: mean = 0.820, std = 0.162
Class2 DSC: mean = 0.882, std = 0.083

Class1 Area:
  Label: mean = 1180.2, std = 318.6
  Pred:  mean = 1470.1, std = 468.5
Class2 Area:
  Label: mean = 891.5, std = 211.8
  Pred:  mean = 890.1, std = 314.8

KS Test for Class1 Area:
  Statistic = 0.2817, p-value = 0.0069
KS Test for Class2 Area:
  Statistic = 0.1127, p-value = 0.7621
